In [6]:
from mso58trans import MSO58transData
from matplotlib import pyplot as plt
import numpy as np
import datetime
import ipywidgets as widgets
from IPython.display import display
from matplotlib import pyplot as plt
from ipyfilechooser import FileChooser

In [43]:

def read_osc():
    
    global t, w
    global now
    now = datetime.datetime.now()
    try:
        handler = MSO58transData('USB::0x0699::0x0530::C043144::INSTR')
        t, w = handler.transfer2byte(1)
        print(f't.shape={t.shape} w.shape={w.shape}')
    except Exception as e:
        print('failed')
        print(e)
    

def save_osc():
    global comment
    filename_w = now.strftime('%Y%m%d_%H%M%S') + '_' + 'wdata_' + comment   + '.npy'
    #filename_w_tes = 'wdata_tes_' + comment + '_' + now.strftime('%Y%m%d_%H%M%S') + '.npy'
    filename_t = now.strftime('%Y%m%d_%H%M%S') + '_' +'tdata_' + comment +  '.npy'
    np.save(filename_w,w)
    #np.save(filename_w_tes,w_tes)
    np.save(filename_t,t)

def var_plot():
    global t, w
    plt.plot(np.var(w[:,:], axis = 0))
    plt.show()


def quadrature_plot():
    global pulseNum
    global hd_idx
    global w, t
    global comment
    global quadrature
    global quadratures
    global now
    global frames

    period = [int(i * w.shape[1]/pulseNum) for i in range(pulseNum)]

    volts = w.T

    varList = []

    frames = w.shape[0]

    fig = plt.figure(figsize = (40,5))
    quadratures = np.zeros((frames, pulseNum))

    ax1 = None
    ax2 = None
    for i in range(pulseNum):
        s = period[i]
        duration = int(w.shape[1]/pulseNum)
        ax1 = fig.add_subplot(2,max(pulseNum,10),i+1, sharey=ax1)
        ax1.plot(volts[s:(s+duration),1])
        ax1.set_title(f'{i}')
        #quadrature = volts[s:(s+duration),:].sum(axis=0)
        quadrature = volts[s+hd_idx,:]
        ax2 = fig.add_subplot(2,max(pulseNum,10),i+1 + max(pulseNum,10), sharey=ax2)
        ax2.scatter(range(w.shape[0]),quadrature,s = 0.01)
        ax2.set_title(f'{i}')
    plt.show()

    for j in range(frames):
        for i in range(pulseNum):
            s = period[i]
            #quadratures[j, i] = volts[s:s+duration, j].sum(axis=0)
            quadratures[j, i] = volts[s+hd_idx, j]

    varList = []

    for i in range(pulseNum):
        s = period[i]
        duration = int(w.shape[1]/pulseNum)
        #quadrature = volts[s:(s+duration),:].sum(axis=0)
        quadrature = volts[s+hd_idx,:]
        varList.append(quadrature.var())

    fig = plt.figure()
    plt.plot(varList)
    plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
    plt.show()

    # ans = 0
    # noiseidx = []
    # for i in range(frames):
    #     if(max(w_tes[i,:]) > 0.03):
    #         noiseidx.append(i)
    # quadratures_delete = np.delete(quadratures, noiseidx, axis=0)
    # print(len(quadratures_delete))
    # filename_hd_delete = 'hd_delete_' + comment + now.strftime('%Y%m%d_%H%M%S') + '.txt'
    # np.savetxt(filename_hd_delete, quadratures_delete)

    fig2 = plt.figure()
    n, bins, patches = plt.hist(quadratures[:,1],bins=100)
    plt.show()

def save_quadratures():
    global comment
    global quadrature
    filename_hd = now.strftime('%Y%m%d_%H%M%S') + 'hd_' + comment + '.txt'
    np.savetxt(filename_hd, quadratures)

def save_hist_img():
    global comment
    global quadratures
    fig2 = plt.figure()
    n, bins, patches = plt.hist(quadratures[:,1],bins=100)
    plt.show()
    fig2.savefig(now.strftime('%Y%m%d_%H%M%S') + 'hist_' + comment + '.png')


def quadrature_sweep_plot():
    global w, t
    global sweep_target_pulse
    frameLength = w.shape[1]
    frames      = w.shape[0]
    period = [int(i * frameLength/pulseNum) for i in range(pulseNum)]
    duration = int(frameLength/pulseNum)
    quadratures = np.zeros((frames, pulseNum))

    volts = np.zeros((frames,pulseNum,duration))
    for i in range(pulseNum):
        s = period[i]
        volts[:,i,:] = w[:,s:(s+duration)]


    fig, ax = plt.subplots(2, 10, sharey="row", figsize = (30,5))
    for i, delta_hd_idx in enumerate(range(-5,5)):
        
        quadratures = volts.reshape((-1,volts.shape[-1]))[:,hd_idx + delta_hd_idx].reshape(volts.shape[:-1])
        ax1 = ax[0,i]
        ax1.scatter(range(quadratures.shape[0]),quadratures[:,sweep_target_pulse],s = 0.01)
        ax1.set_title(f'{delta_hd_idx + hd_idx}')
        ax2 = ax[1,i]
        ax2.hist(quadratures[:,sweep_target_pulse], bins=100)
    plt.show()
    pass

In [44]:


def simple_ui():
    button_clear_output = widgets.Button(description='表示クリア')
    button_read_osc = widgets.Button(description='オシロ読込')
    button_save_osc = widgets.Button(description='オシロ保存')
    button_var_plot = widgets.Button(description='varプロット')
    button_quadrature_plot = widgets.Button(description='quadratureプロット')
    button_quadrature_sweep_plot = widgets.Button(description='hd_idxスイーププロット')
    button_save_quadratures = widgets.Button(description='quadrature保存')
    button_save_hist_img = widgets.Button(description='hist保存')
    filechooser = FileChooser('./')
    button_load_as_wdata_npy = widgets.Button(description='wdata.npyとして読込')
    button_load_as_tdata_npy = widgets.Button(description='tdata.npyとして読込')
    button_input_field = widgets.Button(description='変数反映')

    text_comment = widgets.Text(value='',placeholder='文字を入力',description='comment',disabled=False)
    int_pulseNum = widgets.IntText(value=5,description='pulseNum')
    int_hd_idx = widgets.IntText(value=5,description='hd_idx')
    int_sweep_target_pulse = widgets.IntText(value=0, description='sweep_target_pulse')

    output = widgets.Output(layour={'border': '1px solid black'})
    def wrapped_func_factory(func):
        def new_func(ui_element):
            with output:
                print(f"exec func {func.__name__}")
                func()
                print(f"complete {func.__name__}")
        return new_func
    button_clear_output.on_click(lambda button: output.clear_output(wait=False))
    button_read_osc.on_click(wrapped_func_factory(read_osc))
    button_save_osc.on_click(wrapped_func_factory(save_osc))
    button_var_plot.on_click(wrapped_func_factory(var_plot))
    button_quadrature_plot.on_click(wrapped_func_factory(quadrature_plot))
    button_quadrature_sweep_plot.on_click(wrapped_func_factory(quadrature_sweep_plot))
    button_save_quadratures.on_click(wrapped_func_factory(save_quadratures))
    button_save_hist_img.on_click(wrapped_func_factory(save_hist_img))

    def load_npy_factory(variable_name):
        def load_npy():
            choosed_file_path = filechooser.selected
            global w, t
            if variable_name == 'w':
                w = np.load(choosed_file_path)
                print(f'w.shape={w.shape}')
            elif variable_name == 't':
                t = np.load(choosed_file_path)
                print(f't.shape={t.shape}')
        return load_npy
        
    button_load_as_wdata_npy.on_click(wrapped_func_factory(load_npy_factory('w')))
    button_load_as_tdata_npy.on_click(wrapped_func_factory(load_npy_factory('t')))
    
    def load_input_field():
        global comment
        global pulseNum
        global hd_idx
        global sweep_target_pulse
        comment = text_comment.value
        pulseNum = int_pulseNum.value
        hd_idx = int_hd_idx.value
        sweep_target_pulse = int_sweep_target_pulse.value
    button_input_field.on_click(lambda button: load_input_field())
    
    display(
        widgets.HBox([button_clear_output,button_read_osc,button_save_osc,button_var_plot,button_quadrature_plot,button_quadrature_sweep_plot,button_save_quadratures,button_save_hist_img]),
        widgets.HBox([filechooser, button_load_as_wdata_npy, button_load_as_tdata_npy]),
        widgets.HBox([text_comment,int_pulseNum, int_hd_idx,int_sweep_target_pulse ,button_input_field]),
        output)


In [45]:
simple_ui()

Output()